In [1]:
import pandas as pd
import numpy as np
import pingouin as pg
import plotly.express as px
from itertools import combinations
import plotly.graph_objects as go
import plotly.subplots as sp
import re

# Round 1

In [2]:
df = pd.read_csv('Inputs/round_1_lemann_segmental.csv', header=0, encoding='utf-8')
df = df.drop(['scoreTypeName', 
              'Approx. total length of disease', 'organName', 'organWeight', 
              'organNumSegments'], axis=1)

# Concat patient ID and segment name
df['patient_segment_scoretype'] = df['patientIdentifier'] + ' ' + df['bowelSegmentName'] + ' Lemann'
df['Ulceration'] = df[['Ulcer superficial', 'Ulcer deep']].max(axis=1)
df = df.rename(columns={'segmentIndex': 'segmentalScore'})

lemann = df[['patient_segment_scoretype', 'scorerEmail', 'Mural oedema', 'Fat stranding', 'Ulceration', 'Wall thickening', 
             'Fistula/abscess', 'Inflammation/fibrosis', 'Stricture with pre-stenotic dilatation', 'segmentalScore']]
lemann.set_index('patient_segment_scoretype', inplace=True, drop=True)

# Calculate ICC for features

features = ['Fistula/abscess', 'Stricture with pre-stenotic dilatation', 'Wall thickening', 'Inflammation/fibrosis', 'Mural oedema', 'Ulceration', 'Fat stranding', ]

results = pd.DataFrame()
for feature in features:
    icc = pg.intraclass_corr(data=df, targets='patient_segment_scoretype', raters='scorerEmail',
                            ratings=feature, nan_policy='omit')
    results[feature] = icc.loc[2,:]

results_r1 = results.drop(index=['Type', 'Description', 'F', 'df1', 'df2', 'pval'])

results_r1

,Fistula/abscess,Stricture with pre-stenotic dilatation,Wall thickening,Inflammation/fibrosis,Mural oedema,Ulceration,Fat stranding
ICC,0.59973,0.494517,0.430576,0.363667,0.332068,0.247001,0.126796
CI95%,"[0.54, 0.66]","[0.43, 0.56]","[0.37, 0.5]","[0.3, 0.43]","[0.27, 0.4]","[0.19, 0.31]","[0.08, 0.19]"


# Round 2

In [3]:
df = pd.read_csv('Inputs/round_2_lemann_segmental.csv', header=0, encoding='utf-8')
df = df.drop(['procedureName', 'scoreTypeName', 
              'Approx. total length of disease', 'organName', 'organWeight', 
              'organNumSegments'], axis=1)

# Concat patient ID and segment name
df['patient_segment_scoretype'] = df['patientIdentifier'] + ' ' + df['bowelSegmentName'] + ' Lemann'
df['Ulceration'] = df[['Ulcer superficial', 'Ulcer deep']].max(axis=1)
df = df.rename(columns={'segmentIndex': 'segmentalScore'})

lemann = df[['patient_segment_scoretype', 'scorerEmail', 'Mural oedema', 'Fat stranding', 'Ulceration', 'Wall thickening', 
             'Fistula/abscess', 'Inflammation/fibrosis', 'Stricture with pre-stenotic dilatation', 'segmentalScore']]
lemann.set_index('patient_segment_scoretype', inplace=True, drop=True)

# Calculate ICC for features

features = ['Fistula/abscess', 'Stricture with pre-stenotic dilatation', 'Wall thickening', 'Inflammation/fibrosis', 'Mural oedema', 'Ulceration', 'Fat stranding', ]

results = pd.DataFrame()
for feature in features:
    icc = pg.intraclass_corr(data=df, targets='patient_segment_scoretype', raters='scorerEmail',
                            ratings=feature, nan_policy='omit')
    results[feature] = icc.loc[2,:]

results_r2 = results.drop(index=['Type', 'Description', 'F', 'df1', 'df2', 'pval'])
results_r2

,Fistula/abscess,Stricture with pre-stenotic dilatation,Wall thickening,Inflammation/fibrosis,Mural oedema,Ulceration,Fat stranding
ICC,0.498996,0.248665,0.745256,0.694582,0.654681,0.402288,0.341148
CI95%,"[0.44, 0.56]","[0.19, 0.31]","[0.7, 0.78]","[0.65, 0.74]","[0.61, 0.7]","[0.34, 0.46]","[0.28, 0.4]"


# Combined plot

In [4]:
import plotly.graph_objects as go
import plotly.subplots as sp
import re
import plotly.express as px

def color_to_rgba(color, alpha=1.0):
    # If already rgba, just return
    if color.startswith('rgba'):
        return color
    # If already rgb, add alpha
    if color.startswith('rgb'):
        nums = re.findall(r'\d+', color)
        return f'rgba({nums[0]},{nums[1]},{nums[2]},{alpha})'
    # If hex, convert
    color = color.lstrip('#')
    return f'rgba({int(color[0:2],16)},{int(color[2:4],16)},{int(color[4:6],16)},{alpha})'


# Pick two distinct colors from the same qualitative palette
color_r1 = px.colors.qualitative.Prism[0]
color_r2 = px.colors.qualitative.Prism[1]

# Function to prepare ICC values + asymmetric error bars
def prepare_icc_data(results, features):
    icc_values = results.loc['ICC', features].values
    conf_intervals = results.loc['CI95%', features].values
    errors_lower = [icc_values[i] - conf_intervals[i][0] for i in range(len(icc_values))]
    errors_upper = [conf_intervals[i][1] - icc_values[i] for i in range(len(icc_values))]
    return icc_values, errors_lower, errors_upper

# Extract for both rounds
icc_r1, lower_r1, upper_r1 = prepare_icc_data(results_r1, features)
icc_r2, lower_r2, upper_r2 = prepare_icc_data(results_r2, features)

# Create subplot (1 row, 1 col)
fig = sp.make_subplots(rows=1, cols=1)

# Round 1 bars
fig.add_trace(go.Bar(
    x=features,
    y=icc_r1,
    error_y=dict(
        type='data',
        symmetric=False,
        array=upper_r1,
        arrayminus=lower_r1,
        thickness=2,
        width=5
    ),
    name='Round 1',
    marker=dict(
        color=color_to_rgba(color_r1, 0.55),
        line=dict(color=color_r1, width=2)
    )
), row=1, col=1)

# Round 2 bars
fig.add_trace(go.Bar(
    x=features,
    y=icc_r2,
    error_y=dict(
        type='data',
        symmetric=False,
        array=upper_r2,
        arrayminus=lower_r2,
        thickness=2,
        width=5
    ),
    name='Round 2',
    marker=dict(
        color=color_to_rgba(color_r2, 0.55),
        line=dict(color=color_r2, width=2)
    )
), row=1, col=1)

# Layout adjustments
fig.update_layout(
    xaxis_title="Radiological Features",
    yaxis_title="Feature-specific ICC",
    template="simple_white",
    width=900, height=500,
    barmode='group',   # grouped bars
    legend=dict(
        title="Scoring Round",
        orientation="v",
        # yanchor="bottom",
        # y=1.02,
        # xanchor="center",
        # x=0.5
    )
)
fig.update_yaxes(range=[0, 1], showgrid=True)

fig.show()


In [5]:
# Function to format ICC + CI
def format_icc_ci(row):
    icc = row['ICC']
    ci_lower, ci_upper = row['CI95%']
    return f"{icc:.2f} [{ci_lower:.2f}-{ci_upper:.2f}]"

# Apply for all columns
formatted = []
for col in results_r1.columns:
    icc = results_r1[col].loc['ICC']
    ci_lower, ci_upper = results_r1[col].loc['CI95%']
    formatted.append(f"{icc:.2f} [{ci_lower:.2f}-{ci_upper:.2f}]")

# Put back into a single-row DataFrame
results_r1_summary = pd.DataFrame([formatted], columns=results_r1.columns)
#rename index
results_r1_summary.index = ['Round 1']

# Apply for all columns
formatted = []
for col in results_r2.columns:
    icc = results_r2[col].loc['ICC']
    ci_lower, ci_upper = results_r2[col].loc['CI95%']
    formatted.append(f"{icc:.2f} [{ci_lower:.2f}-{ci_upper:.2f}]")

# Put back into a single-row DataFrame
results_r2_summary = pd.DataFrame([formatted], columns=results_r2.columns)
results_r2_summary.index = ['Round 2']

# combine two dataframes
results_combined = pd.concat([results_r1_summary, results_r2_summary])

results_combined

,Fistula/abscess,Stricture with pre-stenotic dilatation,Wall thickening,Inflammation/fibrosis,Mural oedema,Ulceration,Fat stranding
Round 1,0.60 [0.54-0.66],0.49 [0.43-0.56],0.43 [0.37-0.50],0.36 [0.30-0.43],0.33 [0.27-0.40],0.25 [0.19-0.31],0.13 [0.08-0.19]
Round 2,0.50 [0.44-0.56],0.25 [0.19-0.31],0.75 [0.70-0.78],0.69 [0.65-0.74],0.65 [0.61-0.70],0.40 [0.34-0.46],0.34 [0.28-0.40]
